# Train frame-wise phone classifier

2018-08-22

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np

import tensorflow as tf
import numdifftools as nd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from train import *

## Prepare data

In [2]:
fftlin = np.load('fft_linfilt_pca_reduced.npy')
pca = np.load('fft_linfilt_pca.npy').item()
labels = np.load('labels.npy')
phn2idx = np.load('phn2idx.npy').item()
idx2phn = np.load('idx2phn.npy').item()

In [3]:
X_scaler = StandardScaler().fit(fftlin)
X_std = X_scaler.transform(fftlin)
X_train, X_test, Y_train, Y_test = train_test_split(X_std, labels, test_size=0.2)

In [4]:
# # Divide train/test set
# X_train, X_test, Y_train, Y_test = train_test_split(fftlin, labels, test_size=0.2)

In [5]:
X_train.shape, Y_train.shape

((31021, 40), (31021,))

In [6]:
X_test.shape, Y_test.shape

((7756, 40), (7756,))

### Convert to idx

In [7]:
Y_train_onehot = np.array([phn2idx[phn] for phn in Y_train])
Y_test_onehot = np.array([phn2idx[phn] for phn in Y_test])

In [8]:
max_epoch = 2000
n = 100 # n_display
uq_vowels = list(set(labels))

# Train
total_cost, total_test, moving_avg, final_epoch, params = train_nn(
    (X_train, Y_train_onehot), (X_test, Y_test_onehot), uq_vowels, 
    training_epochs=max_epoch, 
    model_dir=os.path.join('model','test'), n_display=n, verbose=True)

INFO:train:learning_rate: 0.01
INFO:train:training_epochs: 2000
INFO:train:n_display: 100
INFO:train:train_x: (31021, 40), train_y: (31021,)
INFO:train:test_x: (7756, 40), test_y: (7756,)

INFO:train:Hidden size: 500 -> 200 -> 50
INFO:train:final_output_size: 9

INFO:train: 100/2000 train: CE=0.8164, train_acc:0.70, test: CE=0.8355, test_acc:0.69 slop=0.83553
INFO:train: 200/2000 train: CE=0.7213, train_acc:0.73, test: CE=0.8055, test_acc:0.70 slop=-0.02999
INFO:train: 300/2000 train: CE=0.6656, train_acc:0.75, test: CE=0.8020, test_acc:0.70 slop=-0.00352
INFO:train: 400/2000 train: CE=0.6265, train_acc:0.77, test: CE=0.8093, test_acc:0.71 slop=0.00727
INFO:train: 500/2000 train: CE=0.5998, train_acc:0.78, test: CE=0.8281, test_acc:0.71 slop=0.01885
INFO:train: 600/2000 train: CE=0.5758, train_acc:0.78, test: CE=0.8434, test_acc:0.70 slop=0.01523
INFO:train: 700/2000 train: CE=0.5594, train_acc:0.79, test: CE=0.8537, test_acc:0.70 slop=0.01035
INFO:train: 800/2000 train: CE=0.5360, tra

## 해야할 것

- [x] 훈련 accuracy도 추가하기
- [x] hidden size를 늘리고 dropout을 추가하자!  
- [추후] class 형태로 코드 정리하기 (trainmode testmode가 가능하도록)
- [] 테스트 코드 작성하기

## 발견한 것

- Scaling이 1%정도 정확도 향상시킴
- dropout=0.5가 적당히 overfitting막는 듯
- (500 -> 200 -> 50)으로 줄어드는 구조가 정확도 1%정도를 향상시키는 듯

In [20]:
def predict(model_dir, which_epoch, indata, outdata=None):
    tf.reset_default_graph()

    print('model_dir:', model_dir)
    print('final_epoch:', which_epoch)
    print('indata:', indata.shape)
    if outdata is not None:
        print('outdata:', outdata.shape)

    with tf.Session() as sess:
        saver = tf.train.import_meta_graph(
            os.path.join(model_dir, 'model_epoch={}.meta'.format(which_epoch)))
        saver.restore(sess, os.path.join(
            model_dir, 'model_epoch={}'.format(which_epoch)))
        graph = tf.get_default_graph()

        # retrieve variables
        X = graph.get_tensor_by_name('X:0')
        Y = graph.get_tensor_by_name('Y:0')
        keep_prob = graph.get_tensor_by_name('keep_prob:0')
        pred = graph.get_tensor_by_name('prediction:0')
        accuracy = graph.get_tensor_by_name('accuracy:0')
        if outdata is not None:
            cost = graph.get_tensor_by_name('cost/cost:0')
            yhat, error, acc = sess.run([pred, cost, accuracy], feed_dict={
                                  X: indata, Y: outdata, keep_prob:1.0})
            return yhat, error, acc
        else:
            yhat = sess.run(pred, feed_dict={X: indata, keep_prob:1.0})
            return yhat

In [21]:
pred, error, acc = predict('model/test', which_epoch=2000, indata=X_test, outdata=Y_test_onehot)

model_dir: model/test
final_epoch: 2000
indata: (7756, 40)
outdata: (7756,)
INFO:tensorflow:Restoring parameters from model/test/model_epoch=2000


INFO:tensorflow:Restoring parameters from model/test/model_epoch=2000


In [22]:
pred, error.mean(), acc

(array([6, 2, 7, ..., 3, 0, 3]), 0.9990533, 0.7013925)

In [27]:
[(y, idx2phn[i], 'O') if y==idx2phn[i] else (y, idx2phn[i]) for i, y in zip(pred, Y_test)]

[('IY1', 'IY1', 'O'),
 ('AE1', 'AO1'),
 ('UW1', 'UW1', 'O'),
 ('AH1', 'EH1'),
 ('IY1', 'AO1'),
 ('AE1', 'EH1'),
 ('IH1', 'IH1', 'O'),
 ('AO1', 'AO1', 'O'),
 ('EH1', 'AE1'),
 ('IY1', 'IY1', 'O'),
 ('IH1', 'EH1'),
 ('UH1', 'UH1', 'O'),
 ('EH1', 'AE1'),
 ('AH1', 'AH1', 'O'),
 ('AE1', 'AE1', 'O'),
 ('AA1', 'AA1', 'O'),
 ('IH1', 'IH1', 'O'),
 ('IY1', 'IY1', 'O'),
 ('AH1', 'AH1', 'O'),
 ('AO1', 'AH1'),
 ('IH1', 'IH1', 'O'),
 ('EH1', 'AO1'),
 ('UW1', 'UW1', 'O'),
 ('AO1', 'AO1', 'O'),
 ('AE1', 'AE1', 'O'),
 ('AE1', 'AE1', 'O'),
 ('IH1', 'EH1'),
 ('AO1', 'AA1'),
 ('IH1', 'IH1', 'O'),
 ('AE1', 'IH1'),
 ('IY1', 'IY1', 'O'),
 ('AO1', 'AO1', 'O'),
 ('UW1', 'UW1', 'O'),
 ('IY1', 'IY1', 'O'),
 ('UW1', 'UW1', 'O'),
 ('AH1', 'EH1'),
 ('AO1', 'AO1', 'O'),
 ('IY1', 'IY1', 'O'),
 ('IY1', 'IY1', 'O'),
 ('UW1', 'UW1', 'O'),
 ('AO1', 'AO1', 'O'),
 ('AE1', 'AE1', 'O'),
 ('IH1', 'AE1'),
 ('IY1', 'IY1', 'O'),
 ('UW1', 'UW1', 'O'),
 ('AO1', 'AO1', 'O'),
 ('AO1', 'AO1', 'O'),
 ('AE1', 'AH1'),
 ('AO1', 'AO1', 'O'

## Forward 펑션 체크하기

In [28]:
def forward(X, params):
    '''
    This function requires ANN hyper-parameters and sigmoid function
    from outside.
    They were separated for calculating Jacobian matrix (numdifftools)
    '''
    W1, W2, W3, W4 = params['W1'], params['W2'], params['W3'], params['W4']
    b1, b2, b3, b4 = params['b1'], params['b2'], params['b3'], params['b4']
    L1 = relu(np.dot(X, W1) + b1) # Nx300
    L2 = relu(np.dot(L1, W2) + b2) # Nx300
    L3 = relu(np.dot(L2, W3) + b3) # Nx300
    L4 = relu(np.dot(L3, W4) + b4) # Nx300
    return softmax(L4)

In [33]:
params_load = get_param('model/test', which_epoch=2000)
out = forward(X_test, params_load)

model_dir: model/test
which_epoch: 2000
INFO:tensorflow:Restoring parameters from model/test/model_epoch=2000


INFO:tensorflow:Restoring parameters from model/test/model_epoch=2000


In [40]:
pred_load = np.argmax(out, axis=1)

In [42]:
sum(pred_load == pred)

7756

### forward 펑션이 동일한 값을 만드는 것을 확인함 -> forward()를 가져다가 쓰면 됨!!